In [0]:
%sql
-- Create the fact_transactions table if it doesn't exist
CREATE TABLE IF NOT EXISTS gold.fact_transactions (
    TransactionID STRING,
    SRC_TransactionID STRING,
    EncounterID STRING,
    FK_PatientID STRING,
    FK_ProviderID STRING,
    FK_DeptID STRING,
    ICDCode STRING,
    ProcedureCode STRING,
    VisitType STRING,
    ServiceDate DATE,
    PaidDate DATE,
    Amount DOUBLE,
    PaidAmount DOUBLE,
    AmountType STRING,
    ClaimID STRING,
    datasource STRING,
    refreshed_at TIMESTAMP
);



In [0]:
%sql
-- Truncate the fact_transactions table
TRUNCATE TABLE gold.fact_transactions;

In [0]:
%sql

INSERT INTO gold.fact_transactions
SELECT 
    t.TransactionID, 
    t.SRC_TransactionID,
    t.EncounterID,
    CONCAT(t.PatientID, '-', t.datasource) AS FK_PatientID,
    CASE 
        WHEN t.datasource = 'hos-a' THEN CONCAT('H1-', t.providerID) 
        ELSE CONCAT('H2-', t.providerID) 
    END AS FK_ProviderID,
    CONCAT(t.DeptID, '-', t.datasource) AS FK_DeptID,
    t.ICDCode,
    t.ProcedureCode AS CPT_Code,
    t.VisitType,
    t.ServiceDate,
    t.PaidDate,
    t.Amount AS Charge_Amt,
    t.PaidAmount AS Paid_Amt,
    t.AmountType,
    t.ClaimID,
    t.datasource,
    CURRENT_TIMESTAMP()
FROM silver.transactions t
WHERE t.is_current = TRUE AND t.is_quarantined = FALSE;